In [2]:
from model import *
from data_loader import *
from neg_sample import *
import matplotlib.pyplot as plt


print('Starting training...')

# Lightweight training loop for debugging
# num_epochs = 20  # Number of epochs for testing
# batch_size = 64  # Small batch size
# subset_size = len(train_loader.dataset  # Use only a small subset of the dataset
# K = 4  # Number of negative samples

num_epochs = 10  # Number of epochs for testing
batch_size = 32  # Small batch size
K = 4  # Number of negative samples


#model
nrms_model=NRMS(embed_size=300, heads=15, word_embedding_matrix=glove_vectors, attention_dim=200)
# Optimizer for model
optimizer = torch.optim.Adam(nrms_model.parameters(), lr=0.001)

# Initialize a list to store epoch losses
epoch_losses = []

nrms_model.train()
epoch_loss = 0.0


c:\Users\oriol\anaconda3\envs\DL\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset: ebnerd_demo
Loading GloVe vectors from saved file: ../Data/glove_vectors.pt


Statistics for Train Dataset:
Browsed News - Min: 5, Max: 1000, Mean: 182.61, Std: 180.37
Candidate News - Min: 5, Max: 100, Mean: 11.24, Std: 7.92



Statistics for Validation Dataset:
Browsed News - Min: 5, Max: 1000, Mean: 276.52, Std: 219.08
Candidate News - Min: 5, Max: 93, Mean: 12.03, Std: 9.32



RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 22387776000 bytes.

In [ ]:
batch = iter(train_loader)

# Extract batch data
user_histories = batch['browsed_news']  # [batch_size, num_browsed, num_words]
candidate_news = batch['candidate_news']  # [batch_size, num_candidates, num_words]
labels = batch['clicked_idx']  # [batch_size]

print(f"Size of user_histories: {user_histories.size()}")
print(f"Size of candidate_news: {candidate_news.size()}")        

#get click prob
click_prob=nrms_model(user_histories,candidate_news)


In [ ]:
#get click prob of positive samples
#one per batch 
no_batches, no_candidate_news = click_prob.size()
positive_index = torch.arange(no_batches), torch.argmax(labels, dim=1)
positive_sample = click_prob[positive_index]

#get click prob of negative samples 
#more than one per batch
mask=torch.ones_like(click_prob, dtype=torch.bool)
mask[positive_index] = False
negative_samples = click_prob[mask].view(no_batches, -1)

#select K random negative samples
if K > no_candidate_news:
    raise ValueError("K cannot be larger than the size of the tensor.")
    
#use randperm instead of randint so that we dont have repetitions
random_negative_indices = torch.randperm(no_candidate_news)[:K] 
#neg samples for all users (using the same indexes)
negative_samples = click_prob[:, random_negative_indices]  # [batch_size, K]

#compute posterior prob for the possitive sample
exp_pos=torch.exp(positive_sample)  # [batch_size]
exp_neg=torch.exp(negative_samples)  # [batch_size, K]
sum_exp_neg=torch.sum(exp_neg, dim=1)  # [batch_size]
pi_positive=exp_pos/(exp_pos + sum_exp_neg)  # [batch_size]

#average loss across the batch
loss=-torch.log(pi_positive).mean()


In [ ]:


#backward pass and optimization
#clear previous gradients
optimizer.zero_grad()  
loss.backward()   
#update parameters    
optimizer.step()  

#save loss value to calculate for the whole epoch
epoch_loss += loss.item()